### Import dependencies

In [1]:
## Get dependencies ##

import numpy as np
import string
import math
import sys
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
sys.path.append('..')
from GIR import *
import scipy as sp
import pickle
import time
import scipy as sp
from scipy import signal
from scipy.io.idl import readsav
import os
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import glob
import requests
import ftplib
import io
import cmocean
from bs4 import BeautifulSoup
import urllib.request
from io import StringIO, BytesIO
from zipfile import ZipFile
import tqdm
import seaborn as sn
import re

## get SSP emissions
from tools.RCMIP import *

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}
    
## Matplotlib rcparams setup:

matplotlib.rcParams['font.family']='Helvetica'
matplotlib.rcParams['font.size']=11
# matplotlib.rcParams['font.weight']=400

matplotlib.rcParams['image.cmap']='cmo.ice'

matplotlib.rcParams['axes.prop_cycle']=matplotlib.cycler('color',['011936','FF7D00','225560','BFACAA','D72638','788C9B','A33F00','7CAAB0','685655','EB767C'])
matplotlib.rcParams['axes.formatter.limits']=-3,3
# matplotlib.rcParams['axes.labelweight']=300

matplotlib.rcParams['legend.frameon']=False

matplotlib.rcParams['boxplot.whiskers']=(5,95)
matplotlib.rcParams['boxplot.showfliers']=False
matplotlib.rcParams['boxplot.showfliers']=False
matplotlib.rcParams['boxplot.medianprops.color']='black'

matplotlib.rcParams['errorbar.capsize']=5

matplotlib.rcParams['hist.bins']='auto'

plt.rcParams['pdf.fonttype'] = 42

%matplotlib inline

%load_ext line_profiler
%load_ext memory_profiler

/home/leachl/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1762: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)
../tools/RCMIP.py:61: PerformanceWarning: indexing past lexsort depth may impact performance.
  GIR_to_RCMIP_map.loc[RCMIP_to_GIR_map_concs.values(),'RCMIP_concs_unit'] = RCMIP_concs.loc[('World','ssp245')].reindex(RCMIP_to_GIR_map_concs.keys()).loc[:,'Unit'].values#.loc[('World','ssp245',RCMIP_to_GIR_map_concs.keys()),'Unit'].values


In [2]:
FULL_probabilities = pd.read_hdf('../Parameter_Sets/NROY_GWI_informed/FULL_member_selection_probabilities.h5')
GWI_constraint = FULL_probabilities>np.random.random(FULL_probabilities.size)

def get_GWI_ensemble_parameters(ensemble_size=10000):

    gas_param_ensemble = []
    thermal_param_ensemble = []

    gas_param_files = sorted(glob.glob('../Parameter_Sets/NROY_GWI_informed/gas_params*'))
    response_param_files = sorted(glob.glob('../Parameter_Sets/NROY_GWI_informed/response_params*'))

    mems_selected=0
    i=0

    while mems_selected <= ensemble_size:

        unconstrained_data = pd.read_pickle(gas_param_files[i])
        mems_constraint = GWI_constraint.reindex(unconstrained_data.columns.levels[0]).replace(False,np.nan).dropna().index
        gas_param_ensemble += [unconstrained_data.reindex(mems_constraint,axis=1,level=0)]
        unconstrained_data = pd.read_pickle(response_param_files[i])
        thermal_param_ensemble += [unconstrained_data.reindex(mems_constraint,axis=1,level=0)]

        mems_selected += mems_constraint.size

        i+=1

    thermal_param_ensemble = pd.concat(thermal_param_ensemble,axis=1)
    gas_param_ensemble = pd.concat(gas_param_ensemble,axis=1)
    
    return gas_param_ensemble,thermal_param_ensemble

## Construct parameter sets

In [3]:
gas_param_ensemble,thermal_param_ensemble = get_GWI_ensemble_parameters(ensemble_size=50000)

In [4]:
gas_param_ensemble = gas_param_ensemble.reindex(['carbon_dioxide'],axis=1,level=1)
gas_param_ensemble.loc['PI_conc'] = 284

In [5]:
print('# members :',gas_param_ensemble.columns.levels[0].size)

# members : 50328


## Construct scenarios & scenario-specific parameter sets

#### All experiments:
- 1pctCO2
- 1pctCO2-4xext
- 1pctCO2-bgc
- 1pctCO2-cdr
- 1pctCO2-rad
- abrupt-0p5xCO2
- abrupt-2xCO2
- abrupt-4xCO2
- esm-1pct-brch-1000PgC
- esm-1pct-brch-2000PgC
- esm-1pct-brch-750PgC
- esm-bell-1000PgC
- esm-bell-2000PgC
- esm-bell-750PgC
- esm-pi-cdr-pulse
- esm-pi-CO2-pulse

#### Future experiments

- esm-yr2010CO2-cdr-pulse
- esm-yr2010CO2-CO2pulse
- esm-yr2010CO2-control
- esm-yr2010CO2-noemit

In [6]:
# concentration-driven, standard params

## Create the scenarios
conc_driven_experiments = pd.DataFrame(index = np.arange(400), columns = pd.MultiIndex.from_product([['1pctCO2'],['carbon_dioxide']]))

conc_driven_experiments.loc[:399,('1pctCO2','carbon_dioxide')] = 284*1.01**(np.arange(1,401))
conc_driven_experiments.loc[:299,('1pctCO2-4xext','carbon_dioxide')] = 284*1.01**(np.arange(1,301))
conc_driven_experiments.loc[139:,('1pctCO2-4xext','carbon_dioxide')] = 284*4

conc_driven_experiments.loc[:399,('abrupt-4xCO2','carbon_dioxide')] = np.full(400,284*4)
conc_driven_experiments.loc[:399,('abrupt-2xCO2','carbon_dioxide')] = np.full(400,284*2)
conc_driven_experiments.loc[:399,('abrupt-0p5xCO2','carbon_dioxide')] = np.full(400,284*0.5)

# CDRMIP

conc_driven_experiments.loc[:138,('1pctCO2-cdr','carbon_dioxide')] = 284*1.01**(np.arange(1,140))
conc_driven_experiments.loc[139:277,('1pctCO2-cdr','carbon_dioxide')] = 284*4*1.01**(-np.arange(139)-(140-np.log(4)/np.log(1.01)))
conc_driven_experiments.loc[278:,('1pctCO2-cdr','carbon_dioxide')] = 284

In [7]:
## run the conc-driven experiments since some emms are branched
conc_driven_experiment_results = run_GIR(concentrations_in=conc_driven_experiments,gas_parameters=gas_param_ensemble,thermal_parameters=thermal_param_ensemble)

Integrating 6 scenarios, 50328 gas cycle parameter sets, 1 thermal response parameter sets, over ['carbon_dioxide'] forcing agents, between 0 and 399...


100%|██████████| 399/399 [00:58<00:00,  6.79 timestep/s]


In [8]:
## run the additional bgc and rad experiments (rad = cou - bgc)

conc_driven_1pct_bgc = conc_driven_experiments[['1pctCO2']].rename({'1pctCO2':'1pctCO2-bgc'},axis=1,level=0)

gas_params_bgc = gas_param_ensemble.copy()
gas_params_bgc.loc['rT'] = 0

conc_driven_experiment_bgc_results = run_GIR(concentrations_in=conc_driven_1pct_bgc,gas_parameters=gas_params_bgc,thermal_parameters=thermal_param_ensemble)

Integrating 1 scenarios, 50328 gas cycle parameter sets, 1 thermal response parameter sets, over ['carbon_dioxide'] forcing agents, between 0 and 399...


100%|██████████| 399/399 [00:09<00:00, 40.77 timestep/s]


### Historical run (for future)

In [9]:
# ## get historical concs

# CMIP6_conc_ftp = ftplib.FTP('data.iac.ethz.ch','anonymous')
# CMIP6_conc_ftp.cwd('CMIP6/input4MIPs/UoM/GHGConc/CMIP/yr/atmos/UoM-CMIP-1-1-0/GHGConc/gr3-GMNHSH/v20160701')
# CMIP6_ftp_list = [x for x in CMIP6_conc_ftp.nlst() if x[-3:]=='csv']
# WMGHG_concs = pd.DataFrame(dict(zip(['_'.join(x.split('_')[3:-8]) for x in CMIP6_ftp_list],[pd.read_csv('ftp://data.iac.ethz.ch/CMIP6/input4MIPs/UoM/GHGConc/CMIP/yr/atmos/UoM-CMIP-1-1-0/GHGConc/gr3-GMNHSH/v20160701/'+x,usecols=[0,1],index_col=0).iloc[:,0] for x in CMIP6_ftp_list])))
# WMGHG_concs = WMGHG_concs[[x for x in WMGHG_concs.columns if x[-2:]!='eq']] # remove "equivalent" concentrations

# WMGHG_concs['halon1202'] = 0
# WMGHG_concs.loc[1765:2014,'halon1202'] = pd.read_csv('http://www.pik-potsdam.de/~mmalte/rcps/data/RCP45_MIDYEAR_CONCENTRATIONS.DAT',skiprows=38,delim_whitespace=True,index_col=0)['HALON1202'].loc[1765:2014].values

In [10]:
# ## get historical ERFs

# erf_ar6 = pd.read_csv('https://raw.githubusercontent.com/Priestley-Centre/ssp_erf/master/SSPs/ERF_ssp245_1750-2500.csv',index_col=0,dtype=float)
# hist_forc = pd.concat([pd.concat([(erf_ar6.loc[:2014,'total']-erf_ar6.loc[:2014,'co2'])],axis=1,keys=['forcing'])],axis=1,keys=['historical'])
# hist_forc = hist_forc.loc[1850:] - hist_forc.loc[1850]
# hist_concs = pd.concat([WMGHG_concs.loc[1850:,['carbon_dioxide']]],axis=1,keys=['historical'])

In [11]:
# ## historical run

# conc_driven_historical = run_GIR(concentrations_in=hist_concs,forcing_in=hist_forc,gas_parameters=gas_params_bgc,thermal_parameters=thermal_param_ensemble)

In [12]:
# emms driven, single form standard params

emms_driven_experiments = pd.DataFrame(index = np.arange(400), columns = pd.MultiIndex.from_product([['esm-bell-1000PgC'],['carbon_dioxide']]))

emms_driven_experiments.loc[:199,('esm-bell-1000PgC','carbon_dioxide')] = pd.read_csv('http://c4mip.net/fileadmin/user_upload/c4mip/template/ZEC-MIP_Bellcurve_1000PgC.csv',skiprows=4,index_col=0).values.flatten()
emms_driven_experiments.loc[199:,('esm-bell-1000PgC','carbon_dioxide')] = 0
emms_driven_experiments.loc[:199,('esm-bell-750PgC','carbon_dioxide')] = pd.read_csv('http://c4mip.net/fileadmin/user_upload/c4mip/template/ZEC-MIP_Bellcurve_750PgC.csv',skiprows=4,index_col=0).values.flatten()
emms_driven_experiments.loc[199:,('esm-bell-750PgC','carbon_dioxide')] = 0
emms_driven_experiments.loc[:199,('esm-bell-2000PgC','carbon_dioxide')] = pd.read_csv('http://c4mip.net/fileadmin/user_upload/c4mip/template/ZEC-MIP_Bellcurve_2000PgC.csv',skiprows=4,index_col=0).values.flatten()
emms_driven_experiments.loc[199:,('esm-bell-2000PgC','carbon_dioxide')] = 0

emms_driven_experiments.loc[0,('esm-pi-cdr-pulse','carbon_dioxide')] = -100
emms_driven_experiments.loc[1:,('esm-pi-cdr-pulse','carbon_dioxide')] = 0
emms_driven_experiments.loc[0,('esm-pi-CO2-pulse','carbon_dioxide')] = 100
emms_driven_experiments.loc[1:,('esm-pi-CO2-pulse','carbon_dioxide')] = 0

In [13]:
emms_driven_experiment_results = run_GIR(emissions_in=emms_driven_experiments.astype(float),gas_parameters=gas_param_ensemble,thermal_parameters=thermal_param_ensemble)

Integrating 5 scenarios, 50328 gas cycle parameter sets, 1 thermal response parameter sets, over ['carbon_dioxide'] forcing agents, between 0 and 399...


100%|██████████| 399/399 [00:14<00:00, 27.42 timestep/s]


In [14]:
# emms driven, member-specific emissions:

brch_base_1000_1pct = conc_driven_experiment_results['Emissions']['1pctCO2'].apply(lambda x: x * (np.cumsum(x)<1000).astype(int))
brch_base_2000_1pct = conc_driven_experiment_results['Emissions']['1pctCO2'].apply(lambda x: x * (np.cumsum(x)<2000).astype(int))
brch_base_750_1pct = conc_driven_experiment_results['Emissions']['1pctCO2'].apply(lambda x: x * (np.cumsum(x)<750).astype(int))

In [15]:
emms_driven_brch_1000 = run_GIR(emissions_in=brch_base_1000_1pct,gas_parameters=gas_param_ensemble,thermal_parameters=thermal_param_ensemble)
emms_driven_brch_2000 = run_GIR(emissions_in=brch_base_2000_1pct,gas_parameters=gas_param_ensemble,thermal_parameters=thermal_param_ensemble)
emms_driven_brch_750 = run_GIR(emissions_in=brch_base_750_1pct,gas_parameters=gas_param_ensemble,thermal_parameters=thermal_param_ensemble)

Integrating 50328 scenarios, 1 gas cycle parameter sets, 1 thermal response parameter sets, over ['carbon_dioxide'] forcing agents, between 0 and 399...


100%|██████████| 399/399 [00:04<00:00, 93.32 timestep/s] 


Integrating 50328 scenarios, 1 gas cycle parameter sets, 1 thermal response parameter sets, over ['carbon_dioxide'] forcing agents, between 0 and 399...


100%|██████████| 399/399 [00:04<00:00, 98.74 timestep/s] 


Integrating 50328 scenarios, 1 gas cycle parameter sets, 1 thermal response parameter sets, over ['carbon_dioxide'] forcing agents, between 0 and 399...


100%|██████████| 399/399 [00:04<00:00, 99.39 timestep/s] 


In [16]:
### collect results

experiments = ['1pctCO2',
'1pctCO2-4xext',
'1pctCO2-bgc',
'1pctCO2-cdr',
'1pctCO2-rad',
'abrupt-0p5xCO2',
'abrupt-2xCO2',
'abrupt-4xCO2',
'esm-1pct-brch-1000PgC',
'esm-1pct-brch-2000PgC',
'esm-1pct-brch-750PgC',
'esm-bell-1000PgC',
'esm-bell-2000PgC',
'esm-bell-750PgC',
'esm-pi-cdr-pulse',
'esm-pi-CO2-pulse']

In [17]:
member_index = gas_param_ensemble.columns.levels[0]

In [18]:
for exp in conc_driven_experiment_results['Emissions'].columns.levels[0]:

    E = conc_driven_experiment_results['Emissions'][exp].droplevel(axis=1,level=1).reindex(member_index,axis=1)
    E.loc[-1] = 0
    C = pd.DataFrame(data=np.array([conc_driven_experiment_results['C'][exp]['carbon_dioxide'].values]*member_index.size).T,index=np.arange(400),columns=member_index)
    C.loc[-1] = 284
    RF = conc_driven_experiment_results['RF'][exp].xs('Total',axis=1,level=1).reindex(member_index,axis=1)
    RF.loc[-1] = 0
    T = conc_driven_experiment_results['T'][exp].reindex(member_index,axis=1)
    T.loc[-1] = 0
    
    pd.concat([E,C,RF,T],axis=1,keys=['E','C','RF','T']).sort_index().to_hdf(r'../../../FaIR_projection_interactive/idealised_experiments/'+exp+'.h5', key='stage', mode='w')

In [19]:
exp= '1pctCO2-bgc'
E = conc_driven_experiment_bgc_results['Emissions'][exp].droplevel(axis=1,level=1).reindex(member_index,axis=1)
E.loc[-1] = 0
C = pd.DataFrame(data=np.array([conc_driven_experiment_bgc_results['C'][exp]['carbon_dioxide'].values]*member_index.size).T,index=np.arange(400),columns=member_index)
C.loc[-1] = 284
RF = conc_driven_experiment_bgc_results['RF'][exp].xs('Total',axis=1,level=1).reindex(member_index,axis=1)
RF.loc[-1] = 0
T = conc_driven_experiment_bgc_results['T'][exp].reindex(member_index,axis=1)
T.loc[-1] = 0
pd.concat([E,C,RF,T],axis=1,keys=['E','C','RF','T']).sort_index().to_hdf(r'../../../FaIR_projection_interactive/idealised_experiments/'+exp+'.h5', key='stage', mode='w')

In [20]:
exp= '1pctCO2-rad'
E = conc_driven_experiment_results['Emissions']['1pctCO2'].droplevel(axis=1,level=1).reindex(member_index,axis=1) - conc_driven_experiment_bgc_results['Emissions']['1pctCO2-bgc'].droplevel(axis=1,level=1).reindex(member_index,axis=1)
E.loc[-1] = 0
C = pd.DataFrame(data=np.array([conc_driven_experiment_results['C']['1pctCO2']['carbon_dioxide'].values]*member_index.size).T,index=np.arange(400),columns=member_index)
C.loc[-1] = 284
RF = conc_driven_experiment_results['RF']['1pctCO2'].xs('Total',axis=1,level=1).reindex(member_index,axis=1)
RF.loc[-1] = 0
T = conc_driven_experiment_results['T']['1pctCO2'].reindex(member_index,axis=1)
T.loc[-1] = 0
pd.concat([E,C,RF,T],axis=1,keys=['E','C','RF','T']).sort_index().to_hdf(r'../../../FaIR_projection_interactive/idealised_experiments/'+exp+'.h5', key='stage', mode='w')

In [21]:
for exp in emms_driven_experiment_results['Emissions'].columns.levels[0]:

    E = pd.DataFrame(data=np.array([emms_driven_experiment_results['Emissions'][exp]['carbon_dioxide'].values]*member_index.size).T,index=np.arange(400),columns=member_index)
    E.loc[-1] = 0
    C = emms_driven_experiment_results['C'][exp].droplevel(axis=1,level=1).reindex(member_index,axis=1)
    C.loc[-1] = 284
    RF = emms_driven_experiment_results['RF'][exp].xs('Total',axis=1,level=1).reindex(member_index,axis=1)
    RF.loc[-1] = 0
    T = emms_driven_experiment_results['T'][exp].reindex(member_index,axis=1)
    T.loc[-1] = 0
    
    pd.concat([E,C,RF,T],axis=1,keys=['E','C','RF','T']).sort_index().to_hdf(r'../../../FaIR_projection_interactive/idealised_experiments/'+exp+'.h5', key='stage', mode='w')

In [22]:
exp = 'esm-1pct-brch-1000PgC'

E = emms_driven_brch_1000['Emissions'].droplevel(axis=1,level=1).reindex(member_index,axis=1)
C = emms_driven_brch_1000['C'].droplevel(axis=1,level=1).reindex(member_index,axis=1)
RF = emms_driven_brch_1000['RF'].xs('Total',axis=1,level=1).reindex(member_index,axis=1)
T = emms_driven_brch_1000['T'].copy()
T.columns = [x[0] for x in T.columns]
E.loc[-1] = 0
C.loc[-1] = 284
RF.loc[-1] = 0
T.loc[-1] = 0

pd.concat([E,C,RF,T],axis=1,keys=['E','C','RF','T']).sort_index().to_hdf(r'../../../FaIR_projection_interactive/idealised_experiments/'+exp+'.h5', key='stage', mode='w')

exp = 'esm-1pct-brch-2000PgC'

E = emms_driven_brch_2000['Emissions'].droplevel(axis=1,level=1).reindex(member_index,axis=1)
C = emms_driven_brch_2000['C'].droplevel(axis=1,level=1).reindex(member_index,axis=1)
RF = emms_driven_brch_2000['RF'].xs('Total',axis=1,level=1).reindex(member_index,axis=1)
T = emms_driven_brch_2000['T'].copy()
T.columns = [x[0] for x in T.columns]
E.loc[-1] = 0
C.loc[-1] = 284
RF.loc[-1] = 0
T.loc[-1] = 0

pd.concat([E,C,RF,T],axis=1,keys=['E','C','RF','T']).sort_index().to_hdf(r'../../../FaIR_projection_interactive/idealised_experiments/'+exp+'.h5', key='stage', mode='w')

exp = 'esm-1pct-brch-750PgC'

E = emms_driven_brch_750['Emissions'].droplevel(axis=1,level=1).reindex(member_index,axis=1)
C = emms_driven_brch_750['C'].droplevel(axis=1,level=1).reindex(member_index,axis=1)
RF = emms_driven_brch_750['RF'].xs('Total',axis=1,level=1).reindex(member_index,axis=1)
T = emms_driven_brch_750['T'].copy()
T.columns = [x[0] for x in T.columns]
E.loc[-1] = 0
C.loc[-1] = 284
RF.loc[-1] = 0
T.loc[-1] = 0

pd.concat([E,C,RF,T],axis=1,keys=['E','C','RF','T']).sort_index().to_hdf(r'../../../FaIR_projection_interactive/idealised_experiments/'+exp+'.h5', key='stage', mode='w')

### reindexing on emissions / concentrations

In [37]:
def reindex_on_emms(fname):

    dataset = pd.read_hdf(fname)
    index_var='E'
    
#     if not monotonic, don't index
    if (dataset.loc[:,index_var].mean(axis=1)<0).sum() > 0:
        return None

    index = np.linspace(dataset[index_var].cumsum().min().quantile(0.1),dataset[index_var].cumsum().max().quantile(0.9),400)

    members = dataset.columns.levels[1]

    reindexed_list = []
    
    all_vars = ['C','RF','T','E']
    all_vars.remove(index_var)

    for var in all_vars:

        reindex_on_emms = pd.DataFrame(columns=members,index=index)

        for mem in members:
            reindex_on_emms.loc[:,mem] = sp.interpolate.interp1d(dataset.loc[:,(index_var,mem)].cumsum(),dataset.loc[:,(var,mem)],kind='linear',bounds_error=False,fill_value=np.nan)(index)

        reindexed_list += [reindex_on_emms]

    reindexed_list += [pd.DataFrame(data = np.array([index]*members.size).T,columns=members,index=index)]

    reindexed_dataset = pd.concat(reindexed_list,axis=1,keys=all_vars+[index_var]).astype(float)
    
    reindexed_dataset.to_hdf(fname[:-3]+'_reindex-'+index_var+'.h5', key='stage', mode='w')

def reindex_on_var(fname,index_var):

    dataset = pd.read_hdf(fname)
    
    # if not monotonic, don't reindex
    if (dataset.loc[:,index_var].diff().mean(axis=1)<0).sum() > 0:
        return None

    index = np.linspace(np.floor(dataset[index_var].min().quantile(0.1)),dataset[index_var].max().quantile(0.9),400)

    members = dataset.columns.levels[1]

    reindexed_list = []
    
    all_vars = ['C','RF','T','E']
    all_vars.remove(index_var)

    for var in all_vars:

        reindex_on_emms = pd.DataFrame(columns=members,index=index)

        for mem in members:
            reindex_on_emms.loc[:,mem] = sp.interpolate.interp1d(dataset.loc[:,(index_var,mem)],dataset.loc[:,(var,mem)],kind='linear',bounds_error=False,fill_value=np.nan)(index)

        reindexed_list += [reindex_on_emms]

    reindexed_list += [pd.DataFrame(data = np.array([index]*members.size).T,columns=members,index=index)]

    reindexed_dataset = pd.concat(reindexed_list,axis=1,keys=all_vars+[index_var]).astype(float)
    
    reindexed_dataset.to_hdf(fname[:-3]+'_reindex-'+index_var+'.h5', key='stage', mode='w')

In [38]:
experiment_fnames = [x for x in glob.glob('../../../FaIR_projection_interactive/idealised_experiments/*') if not 'reindex' in x]

In [39]:
def reindex_all_exp(exp_fname):
    print('reindexing '+exp_fname)
    reindex_on_emms(exp_fname)
    reindex_on_var(exp_fname,'C')
    reindex_on_var(exp_fname,'RF')

In [40]:
import multiprocessing
P1=multiprocessing.Pool(processes=4)
P1.map(reindex_all_exp,experiment_fnames)
P1.close()

reindexing ../../../FaIR_projection_interactive/idealised_experiments/esm-bell-1000PgC.h5reindexing ../../../FaIR_projection_interactive/idealised_experiments/1pctCO2-4xext.h5reindexing ../../../FaIR_projection_interactive/idealised_experiments/abrupt-0p5xCO2.h5reindexing ../../../FaIR_projection_interactive/idealised_experiments/esm-bell-750PgC.h5



reindexing ../../../FaIR_projection_interactive/idealised_experiments/1pctCO2-bgc.h5
reindexing ../../../FaIR_projection_interactive/idealised_experiments/esm-bell-2000PgC.h5
reindexing ../../../FaIR_projection_interactive/idealised_experiments/abrupt-4xCO2.h5
reindexing ../../../FaIR_projection_interactive/idealised_experiments/esm-pi-CO2-pulse.h5
reindexing ../../../FaIR_projection_interactive/idealised_experiments/esm-1pct-brch-2000PgC.h5
reindexing ../../../FaIR_projection_interactive/idealised_experiments/esm-1pct-brch-750PgC.h5
reindexing ../../../FaIR_projection_interactive/idealised_experiments/abrupt-2xCO2.h5
reindexing ../../../